<a href="https://colab.research.google.com/github/faisalnawazmir/Econometrics-ML_for_Finance/blob/main/ayesha__Fama_5_Factor_split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fama French 5 Factor Model  using split data
2000-2005,2006-2008,2009-2012,2013-2016,2017-2019 (5 splits)

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm 
from tabulate import tabulate
import time
np.random.seed(12345)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [445]:
sb='2017-2019'# split time

In [446]:
#copying data from excel file 25 portfoli
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive
file = r'/gdrive/My Drive/data/FF_factor_ split_data.xlsx'
df = pd.read_excel(file, sheet_name=sb)
df.head()

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


,Date,Rm-Rf,SMB,HML,WML,CMA,RMW,Unnamed: 7,Unnamed: 8,Date.1,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,p17,p18,p19,p20,p21,p22,p23,p24,p25
0,2017-01-01,0.014843,-0.011870,0.016629,-0.051640,-0.064430,-0.039684,NaN,NaN,2017-01-01,0.028101,0.035439,0.029060,0.037174,0.017918,0.018859,0.004814,0.009151,0.015900,0.017504,0.004397,0.005888,-0.002140,-0.008795,-0.011016,0.008446,0.011779,0.010971,0.009399,0.004280,-0.003078,-0.001203,-0.009265,-0.003336,-0.005026
1,2017-01-02,-0.009441,0.027685,0.061939,0.067644,0.024197,0.053202,NaN,NaN,2017-01-02,0.019074,0.036646,0.012164,0.015344,0.013320,-0.004486,-0.032764,-0.025830,-0.011593,-0.012138,0.001202,-0.012525,-0.005592,-0.002651,-0.000181,-0.000252,0.007122,0.007416,0.009492,0.010329,0.007952,0.002430,-0.016385,-0.003015,-0.011921
2,2017-01-03,-0.012673,0.006536,0.020698,0.020230,0.031361,0.040037,NaN,NaN,2017-01-03,-0.003348,-0.001279,0.002225,0.004086,0.018085,0.012355,0.012143,0.012874,0.018864,0.033527,0.032003,0.030343,0.037981,0.042181,0.038091,0.039651,0.040206,0.032929,0.033645,0.022575,-0.004886,0.021728,0.015195,0.011838,0.000796
3,2017-01-04,0.018650,-0.018119,-0.032931,-0.071948,-0.023361,0.011972,NaN,NaN,2017-01-04,0.036349,0.033226,0.030330,0.031842,0.003802,0.001619,0.017445,0.012029,0.015243,0.022545,0.023613,0.014514,0.015315,0.024276,0.026057,0.018759,-0.001152,-0.003580,0.000829,-0.008255,-0.013187,-0.025396,-0.041960,-0.047804,-0.060721
4,2017-01-05,0.020995,0.030408,0.034002,0.037979,0.026583,0.037446,NaN,NaN,2017-01-05,0.147400,0.101580,0.063436,0.048036,0.064690,0.066716,0.045507,0.031786,0.018769,0.008268,0.010219,-0.007003,-0.001505,-0.005142,-0.001723,-0.006371,-0.006311,-0.016328,-0.020156,-0.014850,-0.003571,0.009088,0.006389,-0.001668,-0.004995


In [447]:
# dropping passed columns
df.drop(["Unnamed: 7", "Unnamed: 8","Date.1"], axis = 1, inplace = True)
df.head()
#You must specify the axis argument. default is axis = 0 which is rows columns is axis = 1.

,Date,Rm-Rf,SMB,HML,WML,CMA,RMW,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,p17,p18,p19,p20,p21,p22,p23,p24,p25
0,2017-01-01,0.014843,-0.011870,0.016629,-0.051640,-0.064430,-0.039684,0.028101,0.035439,0.029060,0.037174,0.017918,0.018859,0.004814,0.009151,0.015900,0.017504,0.004397,0.005888,-0.002140,-0.008795,-0.011016,0.008446,0.011779,0.010971,0.009399,0.004280,-0.003078,-0.001203,-0.009265,-0.003336,-0.005026
1,2017-01-02,-0.009441,0.027685,0.061939,0.067644,0.024197,0.053202,0.019074,0.036646,0.012164,0.015344,0.013320,-0.004486,-0.032764,-0.025830,-0.011593,-0.012138,0.001202,-0.012525,-0.005592,-0.002651,-0.000181,-0.000252,0.007122,0.007416,0.009492,0.010329,0.007952,0.002430,-0.016385,-0.003015,-0.011921
2,2017-01-03,-0.012673,0.006536,0.020698,0.020230,0.031361,0.040037,-0.003348,-0.001279,0.002225,0.004086,0.018085,0.012355,0.012143,0.012874,0.018864,0.033527,0.032003,0.030343,0.037981,0.042181,0.038091,0.039651,0.040206,0.032929,0.033645,0.022575,-0.004886,0.021728,0.015195,0.011838,0.000796
3,2017-01-04,0.018650,-0.018119,-0.032931,-0.071948,-0.023361,0.011972,0.036349,0.033226,0.030330,0.031842,0.003802,0.001619,0.017445,0.012029,0.015243,0.022545,0.023613,0.014514,0.015315,0.024276,0.026057,0.018759,-0.001152,-0.003580,0.000829,-0.008255,-0.013187,-0.025396,-0.041960,-0.047804,-0.060721
4,2017-01-05,0.020995,0.030408,0.034002,0.037979,0.026583,0.037446,0.147400,0.101580,0.063436,0.048036,0.064690,0.066716,0.045507,0.031786,0.018769,0.008268,0.010219,-0.007003,-0.001505,-0.005142,-0.001723,-0.006371,-0.006311,-0.016328,-0.020156,-0.014850,-0.003571,0.009088,0.006389,-0.001668,-0.004995


In [448]:
#N = 25  # of firms in an economy, here # of portfolios, here len(data)
T = len(df)  # length of periods # here 1136#df.columns.array
#  data
data = df.loc[:,'p1':'p25']
#data.shape#(T,N)
#x,y=data.shape
#N=y#  # of firms in an economy, here # of portfolios, here len(data)
#T = data.shape[0]
N = data.shape[1]

In [449]:
b_mkt = np.zeros((1,N)) # true loadings for market factor
b_smb = np.zeros((1,N)) # true loadings for size factor
b_hml = np.zeros((1,N)) # true loadings for value factor
b_cma = np.zeros((1,N)) # true loadings for value factor
b_rmw = np.zeros((1,N)) # true loadings for value factor
#b_umd=np.zeros((1,N)) # it is 6th factor 

In [450]:
#  factor returns
mkt = df['Rm-Rf'] # market factor
smb = df['SMB'] # size factor
hml = df['HML'] # value factor
cma = df['CMA'] # market factor
rmw = df['RMW'] # size factor
#umd=df['WML']# umd is growth factor

In [451]:
#data.iloc[:,1]
fctrs = df[["Rm-Rf","SMB","HML","CMA","RMW"]]
fctrs.head()


,Rm-Rf,SMB,HML,CMA,RMW
0,0.014843,-0.011870,0.016629,-0.064430,-0.039684
1,-0.009441,0.027685,0.061939,0.024197,0.053202
2,-0.012673,0.006536,0.020698,0.031361,0.040037
3,0.018650,-0.018119,-0.032931,-0.023361,0.011972
4,0.020995,0.030408,0.034002,0.026583,0.037446


In [452]:
no_factors=fctrs.shape[1]

In [453]:
# Step 1: time series regression firm by firm
#fctrs = np.c_[mkt, smb, hml]
# of factors
b = np.zeros(no_factors)
for i in range(N): # firm-by-firm time series regression
  model = LinearRegression()
  model.fit(fctrs,data.iloc[:,i])
  b = np.c_[b,model.coef_]

b  = np.transpose(b[:,1:])
b.shape
#b should be 0.1, 0.2

(25, 5)

In [454]:
#b is bhat
#b
#b.shape# (N,#factors)
df_bhat = pd.DataFrame(b)
df_bhat.columns = ['b_01','b_02','b_03','b_04','b_05']# change according factos
#print(df_bhat.head())

from google.colab import  drive
df_bhat.to_csv('/gdrive/My Drive/data/bhat_ayesha_5_factors_split_'+sb+'.csv')

In [455]:
# Step 2: cross sectional regression time-by-time
f = np.zeros(no_factors)
for i in range(T): # time-by-time cross sectional regression
  model = LinearRegression()
  model.fit(b,data.iloc[i,:])
  f = np.c_[f,model.coef_]

f = np.transpose(f[:,1:])
df_gamma = pd.DataFrame(f)
df_gamma.columns = ['mkt','smb','hml','cma','rmw']
print(df_gamma.describe())
print('Median estimate:', np.median(f,axis=0))
print('Standard errors (without Shanken correction):', np.std(f,axis=0)/T**.5)

             mkt        smb        hml        cma        rmw
count  36.000000  36.000000  36.000000  36.000000  36.000000
mean   -0.002391   0.002338  -0.006759  -0.006853   0.010412
std     0.061759   0.025376   0.057872   0.060400   0.050974
min    -0.141370  -0.050171  -0.105611  -0.166794  -0.117410
25%    -0.035106  -0.017171  -0.048528  -0.023177  -0.009296
50%    -0.007727   0.000879  -0.016191  -0.002380   0.015234
75%     0.036929   0.016978   0.027222   0.032337   0.039905
max     0.182999   0.077372   0.145358   0.117478   0.128126
Median estimate: [-0.00772712  0.00087922 -0.01619113 -0.00237996  0.01523374]
Standard errors (without Shanken correction): [0.01014919 0.00417023 0.00951039 0.00992594 0.00837685]


## Writing output to the text file

In [456]:
file_output=r'/gdrive/My Drive/data/output_5_factor_split_'+sb+'.txt'
desc=repr(df_gamma.describe())
median_est=repr(np.median(f,axis=0))
std_err=repr(np.std(f,axis=0)/T**.5)

Using the with statement is better practice, it automatically closes the file even if the code encounters an exception. The code will run everything in the indent block then close the file object. 

In [457]:
mu=np.median(f,axis=0)
sigma=(np.std(f,axis=0))/T**.5
tstat=mu/sigma
tstat
t_stats=repr(tstat)
#tstat<1.96

In [458]:
# Write lines to file

with open(file_output, 'w+') as writefile:
    writefile.write("Results of Table  A\n")
    writefile.write(desc+"\n")
    writefile.write("Median estimate:"+"\t"+median_est+"\n")
    writefile.write("Standard errors (without Shanken correction):\t"+std_err+"\n")
    writefile.write("T stats: \t"+t_stats+"\n")
    writefile.write("***********\n\n")


In [459]:
# Check whether write to file

with open(file_output, 'r') as testwritefile:
    print(testwritefile.read())

Results of Table  A
             mkt        smb        hml        cma        rmw
count  36.000000  36.000000  36.000000  36.000000  36.000000
mean   -0.002391   0.002338  -0.006759  -0.006853   0.010412
std     0.061759   0.025376   0.057872   0.060400   0.050974
min    -0.141370  -0.050171  -0.105611  -0.166794  -0.117410
25%    -0.035106  -0.017171  -0.048528  -0.023177  -0.009296
50%    -0.007727   0.000879  -0.016191  -0.002380   0.015234
75%     0.036929   0.016978   0.027222   0.032337   0.039905
max     0.182999   0.077372   0.145358   0.117478   0.128126
Median estimate:	array([-0.00772712,  0.00087922, -0.01619113, -0.00237996,  0.01523374])
Standard errors (without Shanken correction):	array([0.01014919, 0.00417023, 0.00951039, 0.00992594, 0.00837685])
T stats: 	array([-0.76135328,  0.21083278, -1.70246721, -0.23977145,  1.81855193])
***********




In [460]:
f.shape # (T,#factors)# f or df I assume is gamma hat
df_gamma.to_csv('/gdrive/My Drive/data/ayesha_gamma_5_factors_split_'+sb+'.csv')

In [461]:
#df.head()#f and df are same, gamma hat

In [462]:
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
import statsmodels.stats.api as sms

In [463]:

data_a=data.to_numpy()
fctrs_a=fctrs.to_numpy()

In [464]:
# Compute standard error with bootstrapping (Shanken correction unnecessary)
# One simple bootstrapping 

indices = np.random.randint(low=0,high=T, size=T)
new_data = data_a[indices,:]
new_fctrs = fctrs_a[indices,:]
xx = indices[0]
print(xx)
print(new_data[0,:])
print(data_a[xx,:])


12
[ 0.17243952  0.12078118  0.11782312  0.10210069  0.09973716  0.10326357
  0.09905175  0.09769874  0.08539624  0.088562    0.0716488   0.06318648
  0.04292555  0.0379884   0.0387974   0.0380009   0.03704397  0.03477571
  0.02494563  0.01519083 -0.01386608 -0.02554553 -0.01888292 -0.01453388
 -0.04770314]
[ 0.17243952  0.12078118  0.11782312  0.10210069  0.09973716  0.10326357
  0.09905175  0.09769874  0.08539624  0.088562    0.0716488   0.06318648
  0.04292555  0.0379884   0.0387974   0.0380009   0.03704397  0.03477571
  0.02494563  0.01519083 -0.01386608 -0.02554553 -0.01888292 -0.01453388
 -0.04770314]


In [465]:
def efhats(data,fctrs): # function to compute average estimated factor return
  T = data.shape[0]
  N = data.shape[1]
  # Step 1: Time series regression
  x = np.c_[np.ones(T),fctrs]
  M = np.linalg.inv(x.T @ x) @ x.T
  bhat = (M @ data)[1:,:].T

  # Step 2: cross sectional regression
  # Include other control variables here (eg, firm characteristics)
  x = np.c_[np.ones(N),bhat]
  M = np.linalg.inv(x.T @ x) @ x.T
  fhat = (M @ data.T)[1:,:].T
  df = pd.DataFrame(fhat)
  df.columns = ['mkt','smb','hml','cma','rmw']
  E_fhat = np.average(fhat,axis=0)
  return E_fhat 


In [466]:
E_fhat = efhats(data_a,fctrs_a)
simn = 10**3 # of bootstrapping
t = time.time() # bootstrapping start time 
for i in range(simn): 
  indices = np.random.randint(low=0,high=T,size=T) # resample by rows
  new_data = data_a[indices,:] # resampled stock return data
  new_fctrs = fctrs_a[indices,:] # resampled factor return data
  E_fhat = np.c_[E_fhat,efhats(new_data,new_fctrs)]
  # to check bootstrapping status
  if i % 100 == 0: print(i)

print('Simulation duration:', time.time()-t, 'seconds.')
df = pd.DataFrame(E_fhat.T)
df.columns =['mkt','smb','hml','cma','rmw']
print(df.describe())

0
100
200
300
400
500
600
700
800
900
Simulation duration: 1.0311472415924072 seconds.
               mkt          smb          hml          cma          rmw
count  1001.000000  1001.000000  1001.000000  1001.000000  1001.000000
mean     -0.003671     0.001200     0.000040    -0.003643     0.002120
std       0.010892     0.004584     0.009924     0.011712     0.011352
min      -0.040884    -0.014140    -0.036213    -0.036906    -0.031291
25%      -0.011510    -0.001638    -0.006913    -0.011671    -0.005519
50%      -0.003685     0.001247    -0.000296    -0.004027     0.001800
75%       0.003970     0.003973     0.006687     0.003973     0.009011
max       0.031052     0.017748     0.035194     0.048140     0.038236


In [467]:
print('FM regression esimate of expeced factor returns: \n', np.average(E_fhat.T,axis=0))
print('FM regression esimate of standard errors: \n', np.std(E_fhat.T,axis=0))

FM regression esimate of expeced factor returns: 
 [-3.67127010e-03  1.20017613e-03  4.01074385e-05 -3.64250502e-03
  2.11995537e-03]
FM regression esimate of standard errors: 
 [0.01088672 0.00458169 0.00991929 0.0117062  0.01134673]


In [468]:
#bootstrap t stats
mu_boot=np.average(E_fhat.T,axis=0)
sigma_boot=np.std(E_fhat.T,axis=0)
tstat_boot=mu_boot/sigma_boot
tstat_boot

array([-0.33722467,  0.26195045,  0.00404338, -0.31116044,  0.1868341 ])

https://quant.stackexchange.com/questions/35781/how-to-interpret-the-grs-f-test-values
#grs test
https://stackoverflow.com/questions/24074481/fama-macbeth-regression-in-python-pandas-or-statsmodels

In [ ]:
pip install rpy2

In [40]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [172]:
%%R -i df -w 5 -h 5 --units in -r 200
# import df from global environment
# make default figure size 5 by 5 inches with 200 dpi resolution

install.packages("GRS.test", repos='http://cran.us.r-project.org', quiet=TRUE)
library(GRS.test)


In [469]:
%%R -i data,fctrs
GRS.MLtest(data, fctrs)

$GRS.stat
          GRS
[1,] 2.109917

$GRS.pval
           GRS
[1,] 0.1788939

$thetas
         [,1]
[1,] 2.989298

$theta
          [,1]
[1,] 0.4397095

$ratio
          [,1]
[1,] 0.1470945



In [470]:
%%R -o model -o coef  -o pval -o thetas -o theta -o ratio
model<-GRS.MLtest(data, fctrs)
coef<-model$GRS.stat
pval<-model$GRS.pval
thetas<-model$thetas
theta<-model$theta
ratio<-model$ratio

In [471]:
print(model)

$GRS.stat
          GRS
[1,] 2.109917

$GRS.pval
           GRS
[1,] 0.1788939

$thetas
         [,1]
[1,] 2.989298

$theta
          [,1]
[1,] 0.4397095

$ratio
          [,1]
[1,] 0.1470945




W statistic given in (7) of GRS (1989) <DOI:10.2307/1913625>

GRS.stat=GRS test statistic

GRS.pval=its p-value

theta=maximum Sharpe ratio of the K factor portfolios

thetas=slope of the efficient frontier based on all assets

ratio=theta/thetas, proportion of the potential efficiency

References
Gibbons, Ross, Shanken, 1989. A test of the efficiency of a given portfolio, Econometrica, 57,1121-
1152. <DOI:10.2307/1913625>


https://towardsdatascience.com/guide-to-r-and-python-in-a-single-jupyter-notebook-ff12532eb3ba

https://www.linkedin.com/pulse/interfacing-r-from-python-3-jupyter-notebook-jared-stufft/
how to switch R and python


In [472]:
grs_stat=repr(coef)
grs_pval=repr(pval)
grs_thets=repr(thetas)
grs_theta=repr(theta)
grs_ratio=repr(ratio)

In [473]:
#bootstrap t stat results
simulated_desc=repr(df.describe())
median_est_boot=repr(mu_boot)
std_err_boot=repr(sigma_boot)
t_stat_boot=repr(tstat_boot)

In [474]:
# Write lines to file

with open(file_output, 'a') as writefile:
    writefile.write("\n Results of Boot Strapping Table  B\n")
    writefile.write(simulated_desc+"\n")
    writefile.write("FM regression esimate of expeced factor returns:"+"\t"+median_est_boot+"\n")
    writefile.write("FM regression esimate of standard errors:\t"+std_err_boot+"\n")
    writefile.write("Tstats boot strapping: \t"+t_stat_boot+"\n")
    writefile.write("**********\n")
    writefile.write("\t GRS Test:\n")
    writefile.write("**********\n")
    writefile.write("GRS stat:\t"+grs_stat+"\n")
    writefile.write("GRS pvalue:\t"+grs_pval+"\n")
    writefile.write("GRS thetas:\t"+grs_thets+"\n")
    writefile.write("GRS theta:\t"+grs_theta+"\n")
    writefile.write("GRS ratio:\t"+grs_ratio+"\n")

In [444]:
#Erasing the file contents of a text file will result in an empty text file.
# file = open(file_output,"r+")
# file.truncate(0)
# file.close()